# 08 ANOVA - Task 1

Provide a script and html file which calulates the suitable ANOVA to answer the
following research questions (RQ). Please also report the results as a text
conclusion including the test statistic value (F) with degree of freedom, significance
value as well as pairwise comparisions.

Does increasing the bitrate or changing the game (independent variables)
have a significant effect on the video quality (VQ) ratings (dependent variable)?
Please consider all ratings at a resolution of 1080p and a framerate of 60 fps. Use the ratings provided in the gaming video quality dataset.

## Import and Initializing

In [1]:
import numpy as np
import pandas as pd
import scipy

# pip install pingouin
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", context="talk")
cm = sns.diverging_palette(127, 14, s=99, l=55, as_cmap=True)

FIGSIZE = (20,4)

/home/jupyter-groupa/.local/lib/python3.6/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.2, the latest is 0.3.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


## Loading the data

In [15]:
dataset = pd.read_excel(
    "../datasets/DB01_gaming_video_quality_dataset.xlsx",
    usecols=["PID", "Game", "Resolution", "Framerate", "Bitrate", "VQ"],
    dtype={"Bitrate": str}
).dropna()

mask = ((dataset['Resolution'] == 1080) & (dataset['Framerate'] == 60))
dataset = dataset.loc[mask]
assert dataset.groupby("Game").ngroups == 6
assert dataset.groupby("Bitrate").ngroups == 4

## Check general requirements

### Measurement

Independent variables: `Bitrate`, `Game`.<br>
The dependent variable (`VQ`) is measured at the interval level.

### Balance
Remove subjects who do not have measurements for both games.

In [26]:
value_counts = dataset.PID.value_counts()
value_counts.loc[value_counts < 24] = pd.NA
value_counts.dropna(inplace=True)
dataset = dataset.loc[dataset.PID.isin(value_counts.index)]

## One-way ANOVA
![flowchart_one_way_ANOVA](https://pingouin-stats.org/_images/flowchart_one_way_ANOVA.svg)
Source: https://pingouin-stats.org/guidelines.html#id5

## Two-way Repeated Measure ANOVA

In [17]:
result = pg.rm_anova(dataset, dv="VQ", subject="PID", within=["Bitrate", "Game"])
result.style.background_gradient(cmap=cm, subset=["p-unc", "p-GG-corr"])

,Source,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,np2,eps
0,Bitrate,404.142000,3,63,134.714000,278.877000,0.000000,0.000000,0.930000,0.861000
1,Game,12.420000,5,105,2.484000,3.389000,0.007053,0.016365,0.139000,0.717000
2,Bitrate * Game,11.034000,15,315,0.736000,2.079000,0.010746,0.064183,0.090000,0.379000


- `Source`: Name of the within-group factor
- `ddof1`: Degrees of freedom (numerator)
- `ddof2`: Degrees of freedom (denominator)
- `F`: F-value
- `p-unc`: Uncorrected p-value
- `np2`: Partial eta-square effect size
- `eps`: Greenhouse-Geisser epsilon factor (= index of sphericity)
- `p-GG-corr`: Greenhouse-Geisser corrected p-value
- `W-spher`: Sphericity test statistic
- `p-spher`: p-value of the sphericity test
- `sphericity`: sphericity of the data (boolean)

> [...] The default for two-way design is to return both the uncorrected and Greenhouse-Geisser corrected p-values. Note that sphericity test for two-way design are not currently implemented in Pingouin.

Source: https://pingouin-stats.org/generated/pingouin.rm_anova.html?highlight=rm_anova#pingouin.rm_anova

In [28]:
result = pg.pairwise_ttests(dataset, dv="VQ", subject="PID", within=["Game", "Bitrate"])
result.style.background_gradient(cmap=cm, subset=["p-unc"])

,Contrast,Game,A,B,Paired,Parametric,T,dof,Tail,p-unc,BF10,hedges
0,Game,-,Game2,Game5,True,True,2.901000,21.000000,two-sided,0.008541,5.679,0.645000
1,Game,-,Game2,Game1,True,True,4.117000,21.000000,two-sided,0.000491,66.764,0.998000
2,Game,-,Game2,Game6,True,True,3.269000,21.000000,two-sided,0.003668,11.63,0.732000
3,Game,-,Game2,Game3,True,True,1.453000,21.000000,two-sided,0.160921,0.558,0.434000
4,Game,-,Game2,Game4,True,True,0.129000,21.000000,two-sided,0.898689,0.225,0.036000
5,Game,-,Game5,Game1,True,True,1.247000,21.000000,two-sided,0.226004,0.442,0.299000
6,Game,-,Game5,Game6,True,True,0.621000,21.000000,two-sided,0.541379,0.265,0.102000
7,Game,-,Game5,Game3,True,True,-0.495000,21.000000,two-sided,0.625397,0.249,-0.141000
8,Game,-,Game5,Game4,True,True,-2.536000,21.000000,two-sided,0.019239,2.893,-0.418000
9,Game,-,Game1,Game6,True,True,-0.876000,21.000000,two-sided,0.390782,0.314,-0.186000
